[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ThongLai/Credit-Card-Transaction-Fraud-Detection-Using-Explainable-AI/main?urlpath=%2Fdoc%2Ftree%2Fmain.ipynb)

In [1]:
MODEL_PATH = 'architectures/'
DATASET_PATH = 'dataset/'
RANDOM_SEED = 42 # Set to `None` for the generator uses the current system time.

# Importing the necessary packages

In [2]:
# If you are running on `Binder`, then it is no need to set up the packages again
# %pip install -r requirements.txt

# ---OR---

# %pip install tensorflow==2.10.1 numpy==1.26.4 pandas scikit-learn imblearn matplotlib seaborn requests

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# XAI
import shap
import lime
import lime.lime_tabular
from anchor import anchor_tabular

from sklearn.metrics import roc_auc_score, accuracy_score

import os
import time
import utils

np.random.seed(RANDOM_SEED)

Python Version: `3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]`
Base Python location: `C:\Users\LMT\AppData\Local\Programs\Python\Python310`
Current Environment location: `.venv_xai_fraud_detection`

Tensorflow version: `2.10.1`
CUDNN version: `64_8`
CUDA version: `64_112`
Num GPUs Available: 1


# Import test dataset and process data

### Download dataset

In [4]:
utils.download_dataset_from_kaggle('fraudTest.csv')
utils.download_dataset_from_kaggle('fraudTrain.csv')

URL: https://www.kaggle.com/api/v1/datasets/download/kartik2112/fraud-detection/fraudTest.csv
File `dataset/fraudTest.csv` already exists.
URL: https://www.kaggle.com/api/v1/datasets/download/kartik2112/fraud-detection/fraudTrain.csv
File `dataset/fraudTrain.csv` already exists.


### Read data

In [5]:
data_train = pd.read_csv(os.path.join(DATASET_PATH, 'fraudTrain.csv'), index_col=0)
data_test = pd.read_csv(os.path.join(DATASET_PATH, 'fraudTest.csv'), index_col=0)

In [6]:
data_train = utils.feature_engineering(data_train)
X_train, y_train, data_train, transformations = utils.pre_processing(data_train)

data_test = utils.feature_engineering(data_test)
X_test, y_test, data_test, transformations = utils.pre_processing(data_test)

Ordinal-Encoding is applied for `['merchant', 'category', 'gender', 'age_group']`
SMOTE is applied
Ordinal-Encoding is applied for `['merchant', 'category', 'gender', 'age_group']`
SMOTE is applied


# Import pre-trained models and store predictions

In [7]:
models = utils.load_models()


===== MODEL METADATA =====


=== Model: `model_1_Siddhartha_CNN_acc99` ===
Input shape: (None, 13, 1)
Output shape: (None, 1)
Number of layers: 9
Total parameters: 51,873
File size: 0.00 MB
Last modified: Thu Apr 10 16:24:44 2025

--------------------------------------------------


In [ ]:
predictions = {}
for model_name, model in models.items():
    y_predict = model.predict(X_test)
    y_predict_binary = np.round(y_predict).astype(int).squeeze()
    predictions[model_name] = y_predict_binary

19133/34599 [===============>..............] - ETA: 7:50

# Credit Card Fraud Dataset Fields

| Field Name | Description |
|------------|-------------|
| **trans_date_trans_time** | Date and time when transaction occurred |
| **cc_num** | Credit card number of customer |
| **merchant** | Name of merchant where transaction occurred |
| **category** | Category of merchant (e.g., retail, food, etc.) |
| **amt** | Amount of transaction |
| **first** | First name of credit card holder |
| **last** | Last name of credit card holder |
| **gender** | Gender of credit card holder |
| **street** | Street address of credit card holder |
| **city** | City of credit card holder |
| **state** | State of credit card holder |
| **zip** | ZIP code of credit card holder |
| **lat** | Latitude location of credit card holder |
| **long** | Longitude location of credit card holder |
| **city_pop** | Population of credit card holder's city |
| **job** | Occupation of credit card holder |
| **dob** | Date of birth of credit card holder |
| **trans_num** | Transaction number |
| **unix_time** | UNIX timestamp of transaction |
| **merch_lat** | Latitude location of merchant |
| **merch_long** | Longitude location of merchant |
| **is_fraud** | Target class indicating whether transaction is fraudulent (1) or legitimate (0) |

In [ ]:
# %pip install tensorflow==2.10.1 numpy==1.26.4 pandas scikit-learn imblearn matplotlib seaborn

# XAI Techniques

In [ ]:
# %pip install shap lime anchor-exp

For LIME and Anchors, we need to define some characteristics of our data:

In [ ]:
# All features
features = data_test.columns.drop('is_fraud')

# Collect all categorical features
categorical_features = list(data_test.select_dtypes(include=['bool', 'category', 'object']).columns)

# Collect all misclassified entries (For later explaination on why the model predicted them incorrectly)
misclassified_indices = np.where(y_test != y_predict_binary)[0]
print(f"Found {len(misclassified_indices)} misclassified instances")

## SHAP 🎲

### Get SHAP values

Using `DeepExplainer`(specifcifically for neural networks)

**For `DeepExplainer`, we need to create a `background` dataset**: This is because deep neural networks are complex and non-linear, so they require reference points (background samples) to understand how the model normally behaves and accurately calculate feature importance.

Passing the entire training dataset as data will give very accurate expected values, but be unreasonably expensive. The variance of the expectation estimates scale by roughly `1/sqrt(N)` for `N` background data samples.

So 100 samples will give a good estimate, and 1000 samples a very good estimate of the expected values.

In [ ]:
def SHAP(model, X_train, X_test, from_idx, to_idx, background_size=100):
    X_train = np.expand_dims(X_train, axis=-1) if X_train.shape[-1] != 1 else X_train
    X_test = np.expand_dims(X_test, axis=-1) if X_test.shape[-1] != 1 else X_test

    background = X_train[np.random.choice(len(X_train), background_size, replace=False)]

    explainer = shap.DeepExplainer(model, background)

    shap_values = explainer.shap_values(X_test[from_idx:to_idx+1]) # Deep learning models expect 2D input arrays (samples × features), X_test[idx] only returns a 1D array (shape: (n_features,)

    shap_values = shap_values.squeeze()[np.newaxis, ...] if shap_values.shape[0] == 1 else shap_values.squeeze()

    return explainer, shap_values

### Global Interpretability (whole test set)

In [ ]:
# Call the function to obtain SHAP values.
from_idx = misclassified_indices[0]
to_idx = misclassified_indices[0]
# to_idx = len(X_test)-1

explainer, shap_values = SHAP(model, X_train, X_test, from_idx, to_idx, background_size=100)
shap_values.shape

In [ ]:
def get_top_n_features(shap_values, features, n=10):
    mean_shap_values = np.abs(shap_values).mean(axis=0)

    df_shap = pd.DataFrame({
        'feature': features,
        'mean_abs_shap': np.squeeze(mean_shap_values)
    }).set_index('feature')

    df_shap = df_shap.reindex(df_shap['mean_abs_shap'].abs().sort_values(ascending=False).index)

    # Get top n features
    n = 10
    top_n_features = list(df_shap.head(n).index)

    display(df_shap.head(n))

    return top_n_features

top_n_features = get_top_n_features(shap_values, features, n=10)

### Visualization
[SHAP Plots Explained](https://www.youtube.com/playlist?list=PLpoCVQU4m6j9HDOzRBL4nX4eol9DrZ3Kd)

#### Summary Plot

In [ ]:
shap.summary_plot(shap_values, X_test[from_idx:to_idx+1], features)

#### Force Plot

[How to use Shapley Additive Explanations for Black Box Machine Learning Algorithms](https://www.youtube.com/watch?v=7wnG6Wnm2uU)

In [ ]:
# Plot feature contributions for a prediction
shap.initjs()
baseline = explainer.expected_value.numpy()

shap.force_plot(baseline, shap_values, data_test.loc[y_test.index].drop('is_fraud', axis=1).iloc[from_idx:to_idx+1], features)

## LIME 🍋

#### Local interpretable model-agnostic explanations (LIME) [[Paper, 2016]](https://arxiv.org/abs/1602.04938)

> *Interpretable models that are used to explain individual predictions of black box machine learning models (for credit card fraud detection in this project)*

#### **LIME Process in Fraud Detection:**
1. **Select** a transaction of interest (potential fraud case)
2. **Perturb** the transaction features and get black box predictions for these perturbed samples
3. **Generate** a new dataset consisting of perturbed transaction samples and corresponding fraud/legitimate predictions
4. **Train** an interpretable model, weighted by proximity of sampled transactions to the transaction of interest
5. **Interpret** the local model to explain why a specific transaction was flagged as fraudulent

#### **Technical Implementation Details**
For credit card fraud detection, tabular explainers need a training set of transactions . This is because statistics are computed on each feature (such as transaction amount, location, time of day, etc.):
* **Numerical features** (like transaction amount): compute mean and std, discretize into quartiles
* **Categorical features** (like merchant type): compute frequency of each value

This scaling allows meaningful computation of distances between transactions when attributes are not on the same scale.

#### **Key Parameters**
LIME uses an exponential smoothing kernel to define the neighborhood of similar transactions:
* **Small kernel width** = only very similar transactions influence the local model (high precision)
* **Larger kernel width** = transactions with more differences can also influence the model (higher coverage)

When applied to credit card fraud detection, LIME can help explain why a specific transaction was flagged as suspicious by your neural network or other black-box model, making the detection process transparent to financial stakeholders and customers.

[LIME Code Tutorial from original paper authors](https://marcotcr.github.io/lime/tutorials/Tutorial%20-%20continuous%20and%20categorical%20features.html)

#### **Example**

##### **Original Transaction Instance**

```python
transaction = {
    'amt': 1850.75,        # Transaction amount
    'age': 27,             # Customer age
    'dist': 792.3,         # Distance from home location
    'F': 1,                # Female gender
    'M': 0,                # Male gender
    '20 to 30': 1,         # Age bracket
    'AK': 1,               # State (Alaska)
    'shopping_pos': 1,     # Transaction category
}
```

##### **Step 1: Generate Perturbations**

Creating slightly modified versions (perturbations) of the original transaction by adding small random noise based on the feature's statistics (mean and standard deviation):

```python
perturbed_transactions = [
    {    # Perturbed 1 (`AK` changed, `CA` added)
        'amt': 1750.25, 'age': 27, 'dist': 792.3, 
        'F': 1, 'M': 0, '20 to 30': 1, 
        'AK': 0, 'CA': 1, 'shopping_pos': 1
    },
    {    # Perturbed 2 (`shopping_pos` `changed`, `grocery_pos` added)
        'amt': 1850.75, 'age': 27, 'dist': 792.3,
        'F': 1, 'M': 0, '20 to 30': 1, 
        'AK': 1, 'shopping_pos': 0, 'grocery_pos': 1
    },
    {    # Perturbed 3 (`distance`, `age` changed)
        'amt': 1850.75, 'age': 42, 'dist': 156.7,
        'F': 1, 'M': 0, '20 to 30': 0, '40 to 50': 1, 
        'AK': 1, 'shopping_pos': 1
    },
    # ... many more variations
]

# Get predictions from the black box model
predictions = [
    0.35,  # Transaction 1 - lower probability of fraud
    0.42,  # Transaction 2
    0.28,  # Transaction 3
    # ... and so on
]
```

#### **Step 2: Analyze Feature Distributions**
LIME analyzes the distribution of values in the perturbed samples:
```python
# Feature statistics for discretization
amt_stats = {
    'mean': 1523.45,
    'std': 342.87,
    'thresholds': [-0.60, -0.46, -0.32, -0.18, 0.04, 0.26, 0.48]  # normalized
}

dist_stats = {
    'mean': 457.23,
    'std': 389.52,
    'thresholds': [-0.82, -0.51, -0.20, 0.09, 0.41, 0.75, 1.08]  # normalized
}

# ...

For binary categorical features thresholds, LIME typically uses: The percent point function (ppf) of the normal distribution + A small adjustment factor (typically around 0.4 to 0.6) + A small shift constant (often between -0.1 and 0.1)

# Categorical feature analysis
AK_stats = {
    'frequency': 0.03,
    'ppf_calculation': -1.88,  # (ppf(0.03) ≈ -1.88) Inverse of standard normal CDF at 0.03 
    'threshold': -0.06,  # -1.88 * 0.03 - 0.00 ≈ -0.06 (Low adjustment factor (0.03) used due to feature rarity; no shift needed)
}

shopping_pos_stats = {
    'frequency': 0.15,
    'ppf_calculation': -1.04, # (ppf(0.15) ≈ -1.04) Inverse of standard normal CDF at 0.15
    'threshold': -0.33,  # -1.04 * 0.3 + 0.0 ≈ -0.312 (Medium adjustment factor (0.3); no shift applied as base calculation was close to desired scale)
}

grocery_pos_stats = {
    'frequency': 0.22,
    'ppf_calculation': -0.77, # (ppf(0.22) ≈ -0.77) Inverse of standard normal CDF at 0.22
    'threshold': -0.43,  # -0.77 * 0.45 - 0.08 ≈ -0.427 (Medium-high adjustment (0.45) with small negative shift to maintain consistent relationship with `shopping_pos`)
}
```

##### **Step 3: Discretize Continuous Features**
LIME converts continuous features into binary features using thresholds:
```python
# Original transaction (normalized)
normalized_transaction = {
    'amt': -0.51,  # (1850.75 - mean) / std
    'dist': 0.39,  # (792.3 - mean) / std
    # ...other features
}

# Binary features after discretization
binary_features = {
    '-0.60 < amt <= -0.46': 1,  # True
    '0.09 < dist <= 0.41': 1,   # True
    'AK <= -0.06': 1,                    # True
    'F <= 0.91': 1,                     # True
    'shopping_pos <= -0.33': 1,          # True
    # ...
    'amt <= -0.60': 0, 
    '-0.60 < amt <= -0.46': 0, 
    '-0.46 < amt <= -0.32': 0,
    '-0.32 < amt <= -0.18': 0,
    '-0.18 < amt <= 0.04': 0, 
    '0.04 < amt <= 0.26': 0,
    '0.26 < amt <= 0.48': 0,
    'amt > 0.48': 0
    # ... many more binary features (All other features will be 0)
}
```

##### **Step 4: Apply Kernel Weighting**

LIME uses this kernel weighting formula: 

$$\pi_x(z) = \exp\left(-\frac{D(x, z)^2}{\sigma^2}\right)$$

Where:
- $D(x, z)$ is the distance between original transaction $x$ and perturbed sample $z$ (binary distance will be count of features that differ)
- $\sigma$ controls how quickly weight decays with distance

**Perturbed 1**: (`AK` changed, `CA` added)
$$\pi_x(z_1) = \exp\left(-\frac{2^2}{1.5^2}\right) = \exp(-1.78) = 0.17$$

**Perturbed 2**: (`shopping_pos` changed, `grocery_pos` added)
$$\pi_x(z_2) = \exp\left(-\frac{2^2}{1.5^2}\right) = \exp(-1.78) = 0.17$$

**Perturbed 3**: (`distance` bin, `age` bracket changed, new `age` bracket added)
$$\pi_x(z_3) = \exp\left(-\frac{3^2}{1.5^2}\right) = \exp(-4) = 0.02$$


##### **Step 5: Train Local Interpretable Model**

LIME fits a weighted linear model to approximate the black box model locally:

$$g(z) = \beta_0 + \beta_1 z_1 + \beta_2 z_2 + \cdots + \beta_d z_d$$

**Loss Function** (Minimize the weighted squared error):

$$\min_{\beta_0, \beta} \sum_{i=1}^{n} \pi_x(z_i) \left( f(z_i) - \big(\beta_0 + \beta^T z_i\big) \right)^2$$

Where $f(z_i)$ is the black box prediction for perturbed transaction $z_i$.

##### **Step 6: Interpret Feature Contributions**

The coefficients (β) of the linear model show each feature's contribution:

| **Feature**   | **Contribution** | **Interpretation**            |
|---------------|:----------------:|-------------------------------|
| **AK <= -0.06**        |      **+0.73**   | **Strongly indicates fraud**  |
| **-0.60 < amt <= -0.46**       |      **+0.31**   | Moderately indicates fraud    |
| **0.09 < dist <= 0.41**      |      **+0.26**   | Moderately indicates fraud    |
| shopping_pos <= -0.33  |        +0.12     | Slightly indicates fraud      |
| -0.79 < age <= -0.11           |        -0.08     | Slightly indicates legitimate |
| F <= 0.91             |        -0.04     | Minimal impact                |

In [ ]:
kernel_width = np.sqrt(X_train.shape[1]) * 0.75

# LIME explainer
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train,
    feature_names=features,
    class_names=[0, 1],
    categorical_features=categorical_features,
    kernel_width=kernel_width
)

idx = misclassified_indices[1]

print(f'Considering Index: `{idx}`')

# Create a prediction function that returns probabilities for BOTH classes
def model_predict_fn(x):
    preds = model.predict(x)
    two_column_preds = np.concatenate([1 - preds, preds], axis=1)
    return two_column_preds
    
exp = explainer.explain_instance(
    X_test[idx],
    model_predict_fn,
    num_features=10,
)

exp.show_in_notebook()

#### **How to interpret LIME Visualizations**

**Visual Guide:**
* **Left side**: Prediction probability is shown 
* **Blue bars (left)**: Features contributing to "legitimate" prediction
* **Orange bars (right)**: Features contributing to "fraudulent" prediction

**Tabular View:**
* Shows actual value for each feature
* Highlights contribution to each outcome (legitimate or fraudulent)

**Analysis Tips:**
Pay attention to which transaction characteristics most strongly influence the fraud determination. For example:
* Unusually large transaction amount
* Atypical merchant category
* Transaction occurring at unusual time

These insights help financial analysts understand why the model flagged specific transactions, improving both accuracy of manual reviews and overall model transparency.

## Anchors ⚓️

#### High-Precision Model-Agnostic Explanations (Anchors) [Paper, 2018](https://ojs.aaai.org/index.php/AAAI/article/view/11491)

> *Anchors explain individual predictions with simple IF-THEN rules that capture key conditions behind a decision.*

#### **Anchors Process in Fraud Detection:**
1. **Generate Rule Candidates:** Propose simple rules that could explain a model's prediction.
2. **Select Best Anchor:** Identify the rule that best explains the specific transaction.
3. **Validate Precision:** Confirm the rule’s accuracy by testing it on similar cases.
4. **Refine with Search:** Improve the rule using an efficient search algorithm.

#### **Key Insights:**
* **Anchors are IF-THEN rules** that provide clear, high-precision explanations.
* They focus on **accuracy over coverage**: the rule is very reliable when it applies, even if it covers a small group.

[Anchors Code Tutorial from original paper authors](https://github.com/marcotcr/anchor/blob/master/notebooks/Anchor%20on%20tabular%20data.ipynb)

In [ ]:
# Initialize Anchors explainer
explainer = anchor_tabular.AnchorTabularExplainer(
    class_names=['fraud', 'non-fraud'],
    feature_names=features,
    X_train.values,
    categorical_names)

In [ ]:
# Choose a sample for explanation
idx = 100

# Print Prediction
print('Prediction: ', explainer.class_names[model.predict(X_test.values[idx].reshape(1, -1))[0]])

In [ ]:
# Explain the prediction using Anchors
exp = explainer.explain_instance(X_test.values[idx], model.predict, threshold=0.80)

In [ ]:
# Print the prediction, precision, and coverage
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

### How to Interpret

This rule means that if these conditions are met, the model’s fraud prediction is highly reliable. This explanation gives fraud analysts a clear rule they can understand and verify, rather than just a list of contributing factors.

## Single Feature Partial Dependence Plot

[How to Build Shap Single Feature Partial Dependence Plot (PDP Plot)](https://www.youtube.com/watch?v=CgKyAlA-0wA)

In [ ]:
# Dependence plot for specific feature
shap.dependence_plot("amt", shap_values, X_test[from_idx:to_idx+1], features)

## Other Visualization

In [ ]:
fraud_data = data_test[data_test['is_fraud'] == 1]
non_fraud_data = data_test[data_test['is_fraud'] == 0]

In [ ]:
# Statistical analysis of top n features
for feature in top_n_features:
    plt.hist(fraud_data[feature].astype(int), alpha=0.5, label='Fraud', bins=30)
    plt.hist(non_fraud_data[feature].astype(int), alpha=0.5, label='Non-Fraud', bins=30)
    plt.title(f'Distribution of {feature}')
    plt.legend()
    plt.show()

In [ ]:
feature = 'amt'
mean_value = data_test[feature].mean()

fraud_ratio = len(fraud_data[fraud_data[feature] > mean_value]) * 100 / len(data_test[data_test[feature] > mean_value])
legitimate_ratio = 100 - fraud_ratio

plt.pie([fraud_ratio, legitimate_ratio],
        labels=['Fraudulent', 'Legitimate'],
        colors=['crimson', 'lightgreen'],
        autopct='%1.1f%%', startangle=90)
plt.title(f'Distribution of `{feature}` Above Mean of Dataset ({mean_value:.2f})')

In [ ]:
feature = 'shopping_net'

fraud_ratio = len(fraud_data[fraud_data[feature]]) * 100 / len(data_test[data_test[feature]])
legitimate_ratio = 100 - fraud_ratio

plt.pie([fraud_ratio, legitimate_ratio],
        labels=['Fraudulent', 'Legitimate'],
        colors=['crimson', 'lightgreen'],
        autopct='%1.1f%%', startangle=90)
plt.title(f'Distribution of `{feature}` Transactions Above Mean of Dataset ({mean_value:.2f})')

In [ ]:
feature = 'grocery_pos'

fraud_ratio = len(fraud_data[fraud_data[feature]]) * 100 / len(data_test[data_test[feature]])
legitimate_ratio = 100 - fraud_ratio

plt.pie([fraud_ratio, legitimate_ratio],
        labels=['Fraudulent', 'Legitimate'],
        colors=['crimson', 'lightgreen'],
        autopct='%1.1f%%', startangle=90)
plt.title(f'Distribution of `{feature}` Transactions Above Mean of Dataset ({mean_value:.2f})')

In [ ]:
feature = 'gas_transport'

fraud_ratio = len(fraud_data[fraud_data[feature]]) * 100 / len(data_test[data_test[feature]])
legitimate_ratio = 100 - fraud_ratio

plt.pie([fraud_ratio, legitimate_ratio],
        labels=['Fraudulent', 'Legitimate'],
        colors=['crimson', 'lightgreen'],
        autopct='%1.1f%%', startangle=90)
plt.title(f'Distribution of `{feature}` Transactions Above Mean of Dataset ({mean_value:.2f})')

In [ ]:
feature = 'misc_net' # Miscellaneous online transactions

fraud_ratio = len(fraud_data[fraud_data[feature]]) * 100 / len(data_test[data_test[feature]])
legitimate_ratio = 100 - fraud_ratio

plt.pie([fraud_ratio, legitimate_ratio],
        labels=['Fraudulent', 'Legitimate'],
        colors=['crimson', 'lightgreen'],
        autopct='%1.1f%%', startangle=90)
plt.title(f'Distribution of `{feature}` Transactions Above Mean of Dataset ({mean_value:.2f})')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

state_columns = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'FL', 'GA', 'HI', 'IA', 
                'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 
                'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 
                'OR', 'PA', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

states = []
fraud_rates = []
transaction_counts = []
fraud_counts = []

# Calculate metrics for each state
for state in state_columns:
    state_transactions = data_test[data_test[state] == 1]
    total = len(state_transactions)
    
    if total > 0:
        fraud_count = state_transactions['is_fraud'].sum()
        fraud_rate = fraud_count / total
        
        states.append(state)
        fraud_rates.append(fraud_rate)
        transaction_counts.append(total)
        fraud_counts.append(fraud_count)

state_fraud_df = pd.DataFrame({
    'State': states,
    'Fraud_Rate': fraud_rates,
    'Transaction_Count': transaction_counts,
    'Fraud_Count': fraud_counts
})

# Sort by fraud rate descending
state_fraud_df = state_fraud_df.sort_values('Fraud_Rate', ascending=False)

plt.figure(figsize=(12, 10))
ax = sns.barplot(x='Fraud_Rate', y='State', data=state_fraud_df, palette='viridis')

plt.title('Fraud Rate by State', fontsize=16)
plt.xlabel('Fraud Rate', fontsize=12)
plt.ylabel('State', fontsize=12)

for i, v in enumerate(state_fraud_df['Fraud_Rate']):
    ax.text(v + 0.005, i, f"{v:.2%}", va='center')

plt.tight_layout()
plt.show()

# Create a second visualization - bubble chart with fraud rates and transaction volume
plt.figure(figsize=(14, 8))

scatter = plt.scatter(state_fraud_df['Transaction_Count'], 
                     state_fraud_df['Fraud_Rate'], 
                     s=state_fraud_df['Fraud_Count']*5,
                     alpha=0.7,
                     c=state_fraud_df['Fraud_Rate'],
                     cmap='Reds')

for _, row in state_fraud_df.head(5).iterrows():
    plt.annotate(row['State'], 
                (row['Transaction_Count'], row['Fraud_Rate']),
                xytext=(5, 5),
                textcoords='offset points',
                fontweight='bold')

plt.xscale('log')  # Use log scale for better visualization if counts vary widely
plt.grid(True, alpha=0.3)
plt.title('Fraud Rate vs Transaction Volume by State', fontsize=16)
plt.xlabel('Number of Transactions (log scale)', fontsize=12)
plt.ylabel('Fraud Rate', fontsize=12)

cbar = plt.colorbar(scatter)
cbar.set_label('Fraud Rate', rotation=270, labelpad=15)

handles, labels = plt.gca().get_legend_handles_labels()
legend1 = plt.legend(handles, labels, loc="upper left", title="States")

plt.tight_layout()
plt.show()

## Feature Ablation Study

In [ ]:

def feature_ablation_study_global(model, X, y, from_idx, to_idx, selected_features, all_features):
    global replacement_values, ablated_pred
    base_pred = model.predict(X, verbose=0)
    base_auc = roc_auc_score(y, base_pred)

    # Precompute replacement values for each feature to avoid repeated computation.
    replacement_values = {}
    for idx, feature in enumerate(all_features):
        replacement_values[feature] = np.median(X[:, idx])

    print(f"Global (AUC) Feature Ablation Study from index [{from_idx}] to index [{to_idx}]:")
    test_set = X[from_idx:to_idx+1]
    records = []
    for idx, feature in enumerate(all_features):
        X_temp = test_set.copy()
        X_temp[:, idx] = replacement_values[feature]

        ablated_pred = model.predict(X_temp, verbose=0)
        ablated_auc = roc_auc_score(y[from_idx:to_idx+1], ablated_pred)

        impact = ((base_auc - ablated_auc) / base_auc) * 100

        records.append({
            'feature': feature,
            'ablation_auc': ablated_auc,
            'impact_score': impact
        })

    df_results = pd.DataFrame(records).sort_values(by='impact_score', ascending=False, key=abs).reset_index(drop=True)
    df_results['ranking'] = df_results.index+1  # Ranking: 1 denotes the highest impact.
    df_results = df_results[df_results['feature'].isin(selected_features)].reset_index(drop=True)

    return df_results

In [ ]:
df_ablation_global = feature_ablation_study_global(model, X_test, y_test, from_idx, to_idx, top_n_features, features)
df_ablation_global

### Local Interterpretation

In [ ]:
def feature_ablation_single_entry(model, data_entry, selected_features, all_features):
    baseline_values = np.zeros_like(data_entry)

    original_prediction = model.predict(data_entry.reshape(1, -1), verbose=0)

    print(f"Local Feature Ablation Study: ")
    records = []
    for i, feature in enumerate(all_features):
        ablated_entry = data_entry.copy()
        ablated_entry[i] = baseline_values[i]

        ablated_pred = model.predict(ablated_entry.reshape(1, -1), verbose=0) # Compute prediction on the ablated entry

        impact = original_prediction - ablated_pred # Calculate the drop (or change) in prediction

        records.append({
            'feature': feature,
            'ablation_auc': ablated_pred.squeeze(),
            'impact_score': impact.squeeze()
        })

    df_results = pd.DataFrame(records).sort_values(by='impact_score', ascending=False, key=abs).reset_index(drop=True)
    df_results['ranking'] = df_results.index+1  # Ranking: 1 denotes the highest impact.
    df_results = df_results[df_results['feature'].isin(selected_features)].reset_index(drop=True)

    return df_results, original_prediction


In [ ]:
idx = 2
df_ablation_local, original_prob = feature_ablation_single_entry(model, X_test[idx], top_n_features, features)
df_ablation_local

In [ ]:
# Plot feature contributions for a prediction
shap.initjs()
baseline = explainer.expected_value.numpy()

shap.force_plot(baseline, shap_values[idx:idx+1], processed_data.loc[y_test.index].drop('is_fraud', axis=1).iloc[idx:idx+1], features)

In [ ]:
# TO DO

In [ ]:
X_train.shape

## Not Active Codes


```
def feature_ablation_study_local(model, X, y, index, selected_features, all_features):
    global records, base_pred, base_diff, ablated_diff
    base_pred = model.predict(X[index:index+1], verbose=0)
    base_diff = y[index:index+1].to_numpy() - base_pred
    
    # Precompute replacement values for each feature to avoid repeated computation.
    replacement_values = {}
    for idx, feature in enumerate(all_features):
        replacement_values[feature] = np.median(X[:, idx])

    print(f"Local Feature Ablation Study of index [{index}]:")
    test_set = X[index:index+1]
    records = []
    for idx, feature in enumerate(all_features):
        X_temp = test_set.copy()
        X_temp[:, idx] = replacement_values[feature]
        
        ablated_pred = model.predict(X_temp, verbose=0)
        ablated_diff = y[index:index+1].to_numpy() - ablated_pred
        
        impact = np.mean(base_diff - ablated_diff / base_diff)
        
        records.append({
            'feature': feature,
            'ablation_diff': np.mean(ablated_diff),
            'impact_score': impact
        })

    df_results = pd.DataFrame(records).sort_values(by='impact_score', ascending=False, key=abs).reset_index(drop=True)
    df_results['ranking'] = df_results.index+1  # Ranking: 1 denotes the highest impact.
    
    df_results = df_results[df_results['feature'].isin(selected_features)].reset_index(drop=True)
    
    return df_results

df_ablation_local = feature_ablation_study_local(model, X_test, y_test, 2, top_n_features, features)
df_ablation_local

```

## References

Source: https://github.com/siddharthapramanik771/CreditCardFraudDetectionML/blob/main/Credit_card_Fraud_Detection.ipynb

Jira: https://laiminhthong1.atlassian.net/jira/core/projects/CCTFDUX/board

Github Repos: https://github.com/ThongLai/Credit-Card-Transaction-Fraud-Detection-Using-Explainable-AI

Main Dataset: https://www.kaggle.com/datasets/kartik2112/fraud-detection/data